<a href="https://colab.research.google.com/github/jakob-ra/financial_news/blob/master/PL_multi_label_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!pip install pytorch_lightning
!pip install simpletransformers
!pip install nlp

In [2]:
import torch
torch.cuda.get_device_name(0)

'Tesla T4'

In [3]:
import torch
import pytorch_lightning as pl
from simpletransformers.classification import MultiLabelClassificationModel
import pandas as pd
import nlp
import transformers
from transformers import RobertaModel
import gc
import sklearn

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [5]:
DEBUG = False
EPOCHS = 1
BATCH_SIZE = 32
LR = 5e-5
MODEL = 'roberta-large'
SEQ_LENGTH = 128
TEST_SIZE = 0.5
SEED = 42

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
# SDC data
path = '/drive/My Drive/financial_news/'
LABEL_COLS = ['Deal', 'Pending', 'JointVenture', 'Marketing', 'Manufacturing', 'ResearchandDevelopment', 'Licensing', 'Supply', 'Exploration', 'TechnologyTransfer']
NUM_LABELS = len(LABEL_COLS)
df_train = pd.read_csv(path + 'train.csv')
df_test = pd.read_csv(path + 'test.csv')
df_train['text'] = df_train['Text']
df_test['text'] = df_test['Text']
df_train['labels'] = df_train[LABEL_COLS].values.tolist()
df_test['labels'] = df_test[LABEL_COLS].values.tolist()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# small sample
# df_train = df_train.sample(n=1000)
# df_test = df_test.sample(n=1000)

In [8]:
model = MultiLabelClassificationModel('roberta', MODEL, 
                                      use_cuda=True, 
                                      num_labels=NUM_LABELS, 
                                      args={'reprocess_input_data': True, 
                                            'overwrite_output_dir': True,
                                            'num_train_epochs': EPOCHS,
                                            'max_seq_length': SEQ_LENGTH,
                                            'train_batch_size': BATCH_SIZE,
                                            'eval_batch_size': BATCH_SIZE,
                                            'learning_rate': LR,
                                            'save_model_every_epoch': True,
                                            'save_eval_checkpoints': False,
                                            'no_cache': True,
                                            'manual_seed': SEED,
                                            'fp16': True})

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'cl

In [9]:
%%time
model.train_model(df_train, output_dir=path)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)




CPU times: user 57min 42s, sys: 37min 51s, total: 1h 35min 33s
Wall time: 1h 44min 24s


In [10]:
model.eval_model(df_test, verbose=True)

KeyboardInterrupt: ignored

In [ ]:
# small_test = df_test.sample(10)
# preds = model.predict(small_test.Text.values)
# small_test['pred'] = preds[0]

In [27]:
df_test['Text'] = df_test.Text.astype(str)

In [ ]:
df_test['pred'], raw_probs = model.predict(df_test.Text.values)

In [31]:
df_test.to_pickle('df_test_pred.pkl')

In [76]:
df_performance = pd.DataFrame(index=['Precision', 'Recall', 'F1-score', 'Accuracy'], columns=LABEL_COLS).T

In [77]:
for label in range(NUM_LABELS):
  if label == 0: # use all examples to evaluate the 'relevant deal or not' classification
    y_true, y_pred = df_test.labels.map(lambda x: x[label]), df_test.pred.map(lambda x: x[label])
  else: # use only the KB examples as evaluation for relationship classification
    y_true, y_pred = df_test[df_test.Deal == 1].labels.map(lambda x: x[label]), df_test[df_test.Deal == 1].pred.map(lambda x: x[label])
  precision = sklearn.metrics.precision_score(y_true, y_pred)
  recall = sklearn.metrics.recall_score(y_true, y_pred)
  f1_score = sklearn.metrics.f1_score(y_true, y_pred)
  accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
  df_performance.loc[LABEL_COLS[label]] = [precision, recall, f1_score, accuracy]

In [78]:
df_performance

,Precision,Recall,F1-score,Accuracy
Deal,0.997954,0.998543,0.998248,0.998559
Pending,0.937988,0.963693,0.950667,0.969626
JointVenture,0.983384,0.989665,0.986514,0.990009
Marketing,0.863795,0.739044,0.796565,0.932532
Manufacturing,0.877574,0.829638,0.852933,0.952915
ResearchandDevelopment,0.793281,0.749215,0.770619,0.954172
Licensing,0.9128,0.80016,0.852776,0.973097
Supply,0.674482,0.461621,0.548111,0.968336
Exploration,0.776231,0.764447,0.770294,0.989642
TechnologyTransfer,0.693825,0.539665,0.607112,0.949844


In [72]:
df_test[(df_test.ID.str.startswith('http')) & (df_test.pred.map(lambda x: x[0])==1)].Text.values

array(['Indian Oil Corporation Ltd. and NHB Ltd.',
       'GIC Housing Ltd. and Riliance Capital Ltd.',
       'Indian Oil Corporation Ltd. and TMFL Ltd.',
       'Godrej Ind. Ltd Ltd. and Nirma Ltd.',
       'HPCL Corporation Ltd. and Shriram Finance Ltd.', 'Potash Corp.',
       'Chola Finance Ltd. and Tata Capital Ltd.',
       'Indian Oil Corporation Ltd. and Magma Finance Corporation Ltd CP.'],
      dtype=object)

In [80]:
# inference on news from 2012
news_inf = pd.read_pickle(path + 'news_inf.pkl')

In [ ]:
news_inf_small = news_inf.sample(10000)
news_inf_small['pred'], _ = model.predict(news_inf_small.Text.values)

In [83]:
news_inf_small[news_inf_small.pred.map(lambda x: x[0] == 1)].Text.values

array([], dtype=object)

In [ ]:
# Pytorch lightning implementation
def loss_fn(outputs, targets):
    targets = targets.squeeze().type(torch.FloatTensor).cpu()
    outputs = torch.FloatTensor(outputs.cpu())
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

class RobertaMultiLabelClass(torch.nn.Module):
    def __init__(self):
        super(RobertaMultiLabelClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained(MODEL)
        # self.l2 = torch.nn.Dropout(0.3)
        self.l2 = torch.nn.Linear(1024, NUM_LABELS)

    def forward(self, ids, mask):
        _, output_1 = self.l1(ids, attention_mask=mask)
        # output_2 = self.l2(output_1)
        output = self.l2(output_1)
        return output


class RobertaMultiLabelFinetuner(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = RobertaMultiLabelClass().cuda()
        # self.model = MultiLabelClassificationModel('roberta', MODEL, use_cuda=True, num_labels=len(
        #     LABEL_COLS), args={'reprocess_input_data': True, 'overwrite_output_dir': True,
        #     'num_train_epochs': EPOCHS})

    def prepare_data(self):
        tokenizer = transformers.RobertaTokenizer.from_pretrained(
            MODEL,
            do_lower_case=False,
            add_special_tokens=True,
            max_length=SEQ_LENGTH,
            pad_to_max_length=True)

        def _tokenize(x):
            x['input_ids'] = tokenizer.batch_encode_plus(x['comment_text'], max_length=SEQ_LENGTH,
                truncation=True, padding=True)['input_ids']

            return x

        def _prepare_ds(split):
            ds = nlp.Dataset.from_pandas(df)
            # ds = ds.train_test_split(test_size=TEST_SIZE,
            #     shuffle=True, seed=42)
            ds = ds.map(_tokenize, batched=True)
            ds.set_format(type='torch', columns=['input_ids', 'label'], device=DEVICE)
            return ds

        self.train_ds, self.test_ds = map(_prepare_ds, ('train', 'test'))

    def forward(self, input_ids):
        mask = (input_ids != 0).float()
        logits = self.model(input_ids, mask)
        return logits

    def training_step(self, batch, batch_idx):
        logits = self.forward(batch['input_ids'])
        loss = loss_fn(logits, batch['label']).mean()
        return {'loss': loss, 'log': {'train_loss': loss}}

    def validation_step(self, batch, batch_idx):
        logits = self.forward(batch['input_ids'])
        val_loss = loss_fn(logits, batch['label'])
        predicted_vals = logits > 0.5
        val_acc = sklearn.metrics.f1_score(batch['label'].cpu(), predicted_vals.cpu(), average='micro')
        return {'val_loss': val_loss, 'val_acc': val_acc}

    def validation_epoch_end(self, outputs):
        val_loss = torch.stack([x['val_loss'].detach() for x in outputs]).mean()
        val_acc = torch.stack([x['val_acc'].detach() for x in outputs]).mean()
        out = {'val_loss': val_loss, 'val_acc': val_acc}
        return {**out, 'log': out}

    def validation_epoch_end(self, outputs):
        return outputs

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
                self.train_ds,
                batch_size=BATCH_SIZE,
                drop_last=True,
                shuffle=True,
                )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
                self.test_ds,
                batch_size=BATCH_SIZE,
                drop_last=False,
                shuffle=True,
                )
        
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=LR)

class MemoryClearCallback(pl.Callback):

    def on_epoch_end(self, trainer, pl_module):
        print('Clearing Memory')
        gc.collect()
        torch.cuda.empty_cache()

def main():
    model = RobertaMultiLabelFinetuner()
    trainer = pl.Trainer(
        default_root_dir='logs',
        gpus=(1 if DEVICE=='cuda' else 0),
        max_epochs=EPOCHS,
        callbacks=[MemoryClearCallback()],
        # precision=16,
        fast_dev_run=DEBUG,
        logger=pl.loggers.TensorBoardLogger('logs/', name='toxic', version=0),
    )
    trainer.fit(model)


if __name__ == '__main__':
    main()

In [ ]:
# toxic data
LABEL_COLS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
NUM_LABELS = len(LABEL_COLS)
df = pd.read_csv('train_small.csv')
df['label'] = df[LABEL_COLS].values.tolist()
df['text'] = df['comment_text']
df['labels'] = df['label']